<a href="https://colab.research.google.com/github/bbchen33/Insight-project/blob/master/Job_titles_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The notebook is for converting input job titles to output job titles

In [1]:
!pip install --quiet zeugma

In [2]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [3]:
from zeugma.embeddings import EmbeddingTransformer
word2vec = EmbeddingTransformer('word2vec')

[=================================================-] 99.9% 1661.2/1662.8MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
import pickle
from sklearn.linear_model import LogisticRegression
with open('LR_optimized_model', 'rb') as file:
  pretrained_model = pickle.load(file)

The 'clean_indeed_ground_truth_new_titles_and_trained_titles_Feb5_.txt' dataframe is purely for the label encoder used before. If the labelencoder can also just be saved and imported here instead of the dataframe. 

In [0]:
import pandas as pd
ground_truth_combined = pd.read_csv('clean_indeed_ground_truth_new_titles_and_trained_titles_Feb5_.txt', sep = '\t')

In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(ground_truth_combined['official job'].unique())

LabelEncoder()

In [0]:
import re

def remove_symbols(text):
  text = re.sub(r'([0-9]+?)', '', text) 
  text = re.sub(r'(!|"|#|\$|%|&|\'|\(|\)|\*|\+|,|-|\.|/|:|;|<|=|>|\?|@|\[|\\|\]|\^|_|`|{|\||}|~)+', ' ', text)
  return text

def keyword_synonyms_index(job_title_series): # this function will add 'senior' or 'junior' back to the predicted job titles
  keyword_list = ['junior','jr','senior','sr','head']
  common_dict = {'jr':'junior', 'sr':'senior'}
  index_and_kw_dict = {} # index where the keyword occurs
  for title in job_title_series:
    texts = title.split(' ')
    for word in texts:
      if word in keyword_list:
        if word in common_dict:
          index_and_kw_dict[job_title_series.index(title)] = common_dict[word]
        else:
          index_and_kw_dict[job_title_series.index(title)] = word

  return index_and_kw_dict  

def job_title_converter(input_job_list, classifier):
  input_job = pd.Series(input_job_list)
  input_job = input_job.str.lower()
  input_job = input_job.apply(lambda x:remove_symbols(x))
  input_job = input_job.str.strip()
  
  X = word2vec.transform(input_job)
  output = classifier.predict(X)
  probability_output = classifier.predict_proba(X).max(axis =1)
  output = le.inverse_transform(output)
  kw_array = keyword_synonyms_index(input_job_list)
  for index in kw_array:
    output[index] = kw_array[index]+' ' + output[index]

  output_df = pd.DataFrame(columns = ['Input title', 'Output title', 'Probability'])
  output_df['Input title'] = input_job_list
  output_df['Output title'] = output
  output_df['Probability'] = probability_output
  return output_df

# Testing the function.

In [73]:
job_title_converter(['Talent acquis', 'sr recruiting manager', 'hiring staff'], pretrained_model)

,Input title,Output title,Probability
0,Talent acquis,recruiter,0.414830
1,sr recruiting manager,senior recruiter,0.807367
2,hiring staff,recruiter,0.349558
